In [ ]:
# Import Necessary Libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, col
import os
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, DateType

In [ ]:
# Set Java Environment
os.environ['JAVA_HOME'] = "C:/java8"

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("PayrollETL") \
    .config("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

#### Data Extraction

In [ ]:
#Load Master Data
employee_df = spark.read.csv(r'dataset\raw\EmpMaster.csv', header=True, inferSchema=True)
agency_df = spark.read.csv(r'dataset\raw\AgencyMaster.csv', header=True, inferSchema=True)
jobtitle_df = spark.read.csv(r'dataset/raw/TitleMaster.csv', header=True, inferSchema=True)

In [ ]:
# Function to load the multiple payroll datasets and merge them 
def load_data(file_paths):
    payroll_df = [spark.read.csv(file, header=True, inferSchema=True) for file in file_paths]
    merged_df = payroll_df[0]
    for df in payroll_df[1:]:
        merged_df = merged_df.union(df)
    return merged_df.dropDuplicates(["EmployeeID", "FiscalYear"])

In [ ]:
# List all payroll files
payroll_files = [r"dataset\raw\nycpayroll_2020.csv", r"dataset\raw\nycpayroll_2021.csv"]

In [ ]:
# Load and merge payroll data
payroll_df = load_data(payroll_files)